In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from dotenv import load_dotenv
load_dotenv(dotenv_path='/Users/tomwattley/App/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/.env')

True

In [2]:
from trader.fetch_requests import *
from trader.prepare_requests import *
from api_helpers.clients import get_betfair_client, get_s3_client
from api_helpers.helpers.data_utils import print_dataframe_for_testing

In [3]:
betfair_client = get_betfair_client()   
s3_client = get_s3_client()

2025-05-26T15:26:55Z | INFO - Logging into Betfair...
2025-05-26T15:26:55Z | INFO - Logged into Betfair!
2025-05-26T15:26:55Z | INFO - Created new S3 client.


In [4]:
co = betfair_client.get_current_orders()
co

,bet_id,market_id,selection_id,selection_type,execution_status,placed_date,matched_date,average_price_matched,customer_strategy_ref,size_matched,size_remaining,size_lapsed,size_cancelled,size_voided,price_size


In [5]:
betting_data = fetch_betting_data(s3_client, betfair_client)
requests_data = prepare_request_data(betting_data)

2025-05-26T15:27:03Z | INFO - DataFrame loaded from racingapi/today/2025_05_26/trader_data/market_state.parquet.
2025-05-26T15:27:03Z | INFO - DataFrame loaded from racingapi/today/2025_05_26/trader_data/selections.parquet.
2025-05-26T15:27:03Z | INFO - Fetching betting data from today/2025_05_26/trader_data/selections.parquet and today/2025_05_26/trader_data/market_state.parquet
2025-05-26T15:27:03Z | INFO - Found 2 selections
2025-05-26T15:27:04Z | INFO - Found 4 markets
2025-05-26T15:27:04Z | INFO - Found 24 betfair market data
2025-05-26T15:27:04Z | INFO - Found 0 current orders
2025-05-26T15:27:04Z | INFO - Structuring request data


In [7]:
print_dataframe_for_testing(requests_data)

pd.DataFrame({
'race_id':[893919, 893917],
'horse_id':[6674203, 174012],
'horse_name':['Fitzmaurice', 'Maharajas Express'],
'selection_type':['LAY', 'BACK'],
'market_type':['WIN', 'WIN'],
'market_id':['1.244182444', '1.244182438'],
'selection_id':[72693642, 48386184],
'requested_odds':[1.95, 3.8],
'race_time':[pd.Timestamp('2025-05-26 17:40:00+01:00'), pd.Timestamp('2025-05-26 17:05:00+01:00'),],
'minutes_to_race':[133, 98],
'back_price_1':[1.93, 3.85],
'back_price_1_depth':[27.0, 2.0],
'back_price_2':[1.92, 3.8],
'back_price_2_depth':[15.0, 43.0],
'lay_price_1':[1.95, 3.9],
'lay_price_1_depth':[1001.0, 27.0],
'lay_price_2':[1.96, 3.95],
'lay_price_2_depth':[11.0, 7.0],
'eight_to_seven_runners':[False, False],
'short_price_removed_runners':[False, False],
'average_price_matched':[nan, nan],
'size_matched':[0.0, 0.0],
})


In [ ]:
pd.DataFrame(
    {
        "bet_id": ["389581590544", "389581654562", "389582010825", "389582024559"],
        "market_id": ["1.243895856", "1.243895856", "1.243895856", "1.243895856"],
        "selection_id": [71662002, 71662002, 71662002, 71662002],
        "selection_type": ["BACK", "BACK", "BACK", "BACK"],
        "execution_status": [
            "EXECUTION_COMPLETE",
            "EXECUTABLE",
            "EXECUTION_COMPLETE",
            "EXECUTABLE",
        ],
        "placed_date": [
            pd.Timestamp("2025-05-19T19:09:42.000Z"),
            pd.Timestamp("2025-05-19T19:10:18.000Z"),
            pd.Timestamp("2025-05-19T19:13:32.000Z"),
            pd.Timestamp("2025-05-19T19:13:40.000Z"),
        ],
        "matched_date": [
            pd.Timestamp("2025-05-19T19:09:42.000Z"),
            pd.Timestamp("nan"),
            pd.Timestamp("2025-05-19T19:13:32.000Z"),
            pd.Timestamp("nan"),
        ],
        "average_price_matched": [4.2, 0.0, 4.2, 0.0],
        "customer_strategy_ref": ["trader", "UI", "UI", "UI"],
        "size_matched": [5.0, 0.0, 5.0, 0.0],
        "size_remaining": [0.0, 5.0, 0.0, 6.0],
        "size_lapsed": [0.0, 0.0, 0.0, 0.0],
        "size_cancelled": [0.0, 0.0, 0.0, 0.0],
        "size_voided": [0.0, 0.0, 0.0, 0.0],
        "price": [4.1, 5.0, 4.2, 5.0],
        "size": [5.0, 5.0, 5.0, 6.0],
    }
)

pd.DataFrame({
'bet_id':['389581590544', '389581654562', '389582010825', '389582024559'],
'market_id':['1.243895856', '1.243895856', '1.243895856', '1.243895856'],
'selection_id':[71662002, 71662002, 71662002, 71662002],
'selection_type':['BACK', 'BACK', 'BACK', 'BACK'],
'execution_status':['EXECUTION_COMPLETE', 'EXECUTABLE', 'EXECUTION_COMPLETE', 'EXECUTABLE'],
'placed_date':[pd.Timestamp('2025-05-19T19:09:42.000Z'), pd.Timestamp('2025-05-19T19:10:18.000Z'), pd.Timestamp('2025-05-19T19:13:32.000Z'), pd.Timestamp('2025-05-19T19:13:40.000Z'),],
'matched_date':[pd.Timestamp('2025-05-19T19:09:42.000Z'), pd.Timestamp('nan'), pd.Timestamp('2025-05-19T19:13:32.000Z'), pd.Timestamp('nan'),],
'average_price_matched':[4.2, 0.0, 4.2, 0.0],
'customer_strategy_ref':['trader', 'UI', 'UI', 'UI'],
'size_matched':[5.0, 0.0, 5.0, 0.0],
'size_remaining':[0.0, 5.0, 0.0, 6.0],
'size_lapsed':[0.0, 0.0, 0.0, 0.0],
'size_cancelled':[0.0, 0.0, 0.0, 0.0],
'size_voided':[0.0, 0.0, 0.0, 0.0],
})


In [96]:
sf = pd.DataFrame(
    {
        "race_id": [1, 2],
        "race_time": [
            pd.Timestamp("2020-01-01 12:00:00"),
            pd.Timestamp("2020-01-01 13:00:00"),
        ],
        "race_date": [
            pd.Timestamp("2020-01-01"),
            pd.Timestamp("2020-01-01"),
        ],
        "horse_id": [1, 2],
        "horse_name": ["Horse A", "Horse B"],
        "selection_type": ["BACK", "LAY"],
        "market_type": [
            "WIN",
            "WIN",
        ],
        "market_id": ["1", "2"],
        "selection_id": [1, 2],
        "requested_odds": [3, 4],
    }
)

In [97]:
co = pd.DataFrame(
    {
        "bet_id": [1, 2, 3, 4],
        "market_id": ["1", "1", "2", "1"],
        "selection_id": [1, 1, 2, 1],
        "selection_type": ["BACK", "BACK", "LAY", "BACK"],
        "execution_status": [
            "EXECUTION_COMPLETE",
            "EXECUTABLE",
            "EXECUTABLE",
            "EXECUTION_COMPLETE",
        ],
        "placed_date": [pd.Timestamp("2020-01-01")] * 4,
        "matched_date": [pd.Timestamp("2020-01-01")] * 4,
        "average_price_matched": [3, np.nan, 2, 3.5],
        "size_matched": [5, 0, 2, 10],
        "size_remaining": [0, 5, 3, 0],
        "customer_strategy_ref": ["mvp"] * 4,
    }
)

In [98]:
df = pd.merge(
    sf,
    co,
    on=["market_id", "selection_id", "selection_type"],
    how="left",
)

In [99]:
df

,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,selection_id,requested_odds,bet_id,execution_status,placed_date,matched_date,average_price_matched,size_matched,size_remaining,customer_strategy_ref
0,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,1,EXECUTION_COMPLETE,2020-01-01,2020-01-01,3.0,5,0,mvp
1,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,2,EXECUTABLE,2020-01-01,2020-01-01,NaN,0,5,mvp
2,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,4,EXECUTION_COMPLETE,2020-01-01,2020-01-01,3.5,10,0,mvp
3,2,2020-01-01 13:00:00,2020-01-01,2,Horse B,LAY,WIN,2,2,4,3,EXECUTABLE,2020-01-01,2020-01-01,2.0,2,3,mvp


In [100]:
matched = df[df["execution_status"] == "EXECUTION_COMPLETE"].assign(
    matched_status="matched"
)
unmatched = df[df["execution_status"] != "EXECUTION_COMPLETE"].assign(
    matched_status="unmatched"
)

In [101]:
group_cols = ["race_id", "market_id", "selection_type", "selection_id"]
matched["payoff"] = matched["size_matched"] * matched["average_price_matched"]
matched["total_stake"] = matched.groupby(group_cols)["size_matched"].transform("sum")
matched["total_odds"] = matched.groupby(group_cols)["payoff"].transform("sum")
matched["ave_odds"] = (matched["total_odds"] / matched["total_stake"]).round(2)
matched = matched.drop_duplicates(subset=group_cols)
matched

,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,selection_id,requested_odds,...,matched_date,average_price_matched,size_matched,size_remaining,customer_strategy_ref,matched_status,payoff,total_stake,total_odds,ave_odds
0,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,...,2020-01-01,3.0,5,0,mvp,matched,15.0,15,50.0,3.33


In [104]:
matched = matched[
    [
        "race_id",
        "race_time",
        "race_date",
        "horse_id",
        "horse_name",
        "selection_type",
        "market_type",
        "market_id",
        "selection_id",
        "requested_odds",
        "placed_date",
        "matched_date",
        "matched_status",
        "size_remaining",
        "total_stake",
        "ave_odds",
    ]
].rename(columns={"total_stake": "size_matched", "ave_odds": "average_price_matched"})

In [105]:
unmatched = unmatched[matched.columns]

In [106]:
unmatched

,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,selection_id,requested_odds,placed_date,matched_date,matched_status,size_remaining,size_matched,average_price_matched
1,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,2020-01-01,2020-01-01,unmatched,5,0,NaN
3,2,2020-01-01 13:00:00,2020-01-01,2,Horse B,LAY,WIN,2,2,4,2020-01-01,2020-01-01,unmatched,3,2,2.0


In [107]:
data = pd.concat([matched, unmatched])
data

,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,selection_id,requested_odds,placed_date,matched_date,matched_status,size_remaining,size_matched,average_price_matched
0,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,2020-01-01,2020-01-01,matched,0,15,3.33
1,1,2020-01-01 12:00:00,2020-01-01,1,Horse A,BACK,WIN,1,1,3,2020-01-01,2020-01-01,unmatched,5,0,NaN
3,2,2020-01-01 13:00:00,2020-01-01,2,Horse B,LAY,WIN,2,2,4,2020-01-01,2020-01-01,unmatched,3,2,2.00


In [11]:
now_timestamp = get_uk_time_now()

folder = f"today/{now_timestamp.strftime('%Y_%m_%d')}"

selections_file_path = f"{folder}/selections.parquet"
fully_matched_bets_file_path = f"{folder}/fully_matched_bets.parquet"
cashed_out_bets_file_path = f"{folder}/cashed_out_bets_bets.parquet"
invalidated_bets_file_path = f"{folder}/invalidated_bets.parquet"
market_state_file_path = f"{folder}/market_state.parquet"

In [12]:
fully_matched_bets = s3_client.fetch_data(fully_matched_bets_file_path)

2025-05-19T20:31:39Z | INFO - DataFrame loaded from racingapi/today/2025_05_19/fully_matched_bets.parquet.


In [13]:
fully_matched_bets

,race_time,race_id,horse_id,horse_name,selection_type,market_type,average_price_matched,size_matched
0,2025-05-19 20:30:00+01:00,894178,6673304,Natsumi,BACK,WIN,4.2,10.0
1,2025-05-19 21:00:00+01:00,894176,166176,Novak,BACK,WIN,5.6,5.0


In [14]:
selections = s3_client.fetch_data(selections_file_path)
selections

2025-05-19T20:32:05Z | INFO - DataFrame loaded from racingapi/today/2025_05_19/selections.parquet.


,id,timestamp,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,selection_id,in_dutch,bet_group_id,request_id,requested_odds,valid,invalidated_at,invalidated_reason
0,5985382e-5fb3-4e91-bd81-fba1b730e9de,2025-05-19 19:05:56.123000+00:00,894178,2025-05-19 20:30:00,2025-05-19,6673304,Natsumi,BACK,WIN,1.243895856,71662002,False,8912f41a-69a2-40eb-82f7-1b4383931b3a,478222324594608,4.0,True,None,None
1,bd740251-8ded-4783-a73c-8a1fa24d42d0,2025-05-19 19:27:53.305000+00:00,894176,2025-05-19 21:00:00,2025-05-19,166176,Novak,BACK,WIN,1.243895863,53180,False,05f5468e-d880-4339-bd5c-5a811e204af5,8837239680,5.7,True,None,None
2,f89e052f-f0dd-4166-9976-a31f5a5a5313,2025-05-19 19:28:42.596000+00:00,894176,2025-05-19 21:00:00,2025-05-19,166176,Novak,BACK,WIN,1.243895863,53180,False,c8cd9e09-3f02-435b-b42a-0dfe8ab47195,8837239680,5.4,True,None,None


In [6]:
from pathlib import PosixPath

In [7]:
a = PosixPath(
    "/Users/tomwattley/Code/python/racing-api-project/racing-api-project/libraries/api-helpers/src/api_helpers/helpers"
)

In [16]:
path = a.parent.parent.parent.parent.parent / "apps" / "racing-etl" / ".env"

In [17]:
path

PosixPath('/Users/tomwattley/Code/python/racing-api-project/racing-api-project/apps/racing-etl/.env')

In [ ]:
"/Users/tomwattley/Code/python/racing-api-project/racing-api-project/apps/racing-etl/.env"
"/Users/tomwattley/Code/python/racing-api-project/racing-api-project/apps/racing_etl/.env"